In [1]:
pip install pandas

In [2]:
import pandas as pd

# Function to read CSV file
def read_csv(csv_file):
    df = pd.read_csv(csv_file)
    return df

# Example usage:
csv_file_path = '/content/leads_main.csv'
leads_df = read_csv(csv_file_path)

# Display the first 5 rows
print(leads_df.head())

     LeadID First Name Last Name                          Email       Phone  \
0  LEAD1000         Md     Tawaf           mdtawaf742@gmail.com  1724889481   
1  LEAD1001     Turjoy       Ewu            turjoyewu@gmail.com  1724889432   
2  LEAD1002       Tara      Coma             tmcom098@gmail.com  1724889490   
3  LEAD1003      Hasan  Mohammed  hasanmohammedturjoy@gmail.com  1724889874   
4  LEAD1004      Shifa    Rahman      shifa.rahman989@gmail.com  1724889431   

  Interest Category  Lead Score Buyer Persona  
0            Retail           2    Individual  
1           Finance           9    Enterprise  
2         Marketing           7           SMB  
3       Real Estate           3           SMB  
4            Retail           8    Non-Profit  


In [3]:
pip install transformers

In [4]:
import pandas as pd
from transformers import pipeline

# Load the HuggingFace GPT-2 or GPT-Neo model
generator = pipeline('text-generation', model='gpt2')

# Function to generate email for each lead using HuggingFace model
def generate_email_with_huggingface(lead):
    prompt = f"Generate a personalized email for {lead['First Name']} {lead['Last Name']}, who is interested in {lead['Interest Category']}. The email should be professional and engaging."

    # Generate the email using the model
    email = generator(prompt, max_length=150, num_return_sequences=1)[0]['generated_text']
    return email

# Function to generate emails for all leads in the dataset
def generate_emails_for_all_leads(csv_file):
    leads_df = pd.read_csv(csv_file)

    # Create a new column for generated emails
    leads_df['Generated Email'] = leads_df.apply(generate_email_with_huggingface, axis=1)

    # Save the updated CSV with generated emails
    leads_df.to_csv('generated_emails_with_status.csv', index=False)
    print("Emails generated and saved to 'generated_emails_with_status.csv'!")

# Example usage
csv_file_path = '/content/leads_main.csv'
generate_emails_for_all_leads(csv_file_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transform

Emails generated and saved to 'generated_emails_with_status.csv'!


In [5]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import time

# Function to send email securely via Gmail SMTP
def send_email_secure(to, subject, body, email, app_password):
    # Create a multipart message
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = to
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Using Gmail's SMTP server with SSL
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(email, app_password)
            server.sendmail(msg['From'], [msg['To']], msg.as_string())
        return "Sent"
    except Exception as e:
        return f"Failed: {str(e)}"

# Function to send emails to all leads and update CSV with status
def send_emails_and_update_csv(csv_file, email, app_password):
    leads_df = pd.read_csv(csv_file)

    # Iterate over each lead and send an email
    for index, lead in leads_df.iterrows():
        subject = f"Hello {lead['First Name']}, We have an offer for you!"
        body = lead['Generated Email']

        # Send email and update the status in CSV
        status = send_email_secure(lead['Email'], subject, body, email, app_password)
        leads_df.at[index, 'Email Status'] = status

        # Adding a small delay between emails to avoid rate limiting or spamming
        time.sleep(1)

    # Save updated CSV with email status
    leads_df.to_csv('updated_leads_with_status.csv', index=False)
    print("Emails sent and status updated in 'updated_leads_with_status.csv'!")

# Example usage
csv_file_path = '/content/generated_emails_with_status.csv'
email = '2021-2-60-072@std.ewubd.edu'
app_password = 'tnxw uvnk uwrg axvv'
send_emails_and_update_csv(csv_file_path, email, app_password)

Emails sent and status updated in 'updated_leads_with_status.csv'!


In [6]:
import pandas as pd

# Function to generate the campaign summary report
def generate_campaign_summary(csv_file):
    # Read the updated CSV file
    leads_df = pd.read_csv(csv_file)

    # Calculate the statistics for the campaign
    total_leads = len(leads_df)
    emails_sent = len(leads_df[leads_df['Email Status'] == 'Sent'])
    emails_failed = len(leads_df[leads_df['Email Status'] != 'Sent'])

    # Create the summary content
    summary = f"""
    # Campaign Summary

    - **Total Leads Processed**: {total_leads}
    - **Emails Sent Successfully**: {emails_sent}
    - **Emails Failed**: {emails_failed}

    ## Key Insights:
    - Most of the emails were successfully sent, with only a few failures.
    - Consider retrying the failed emails or investigating the issues (e.g., invalid email addresses, network issues).

    ## Recommendations:
    - Review the failed email logs to ensure they are delivered successfully in the next campaign.
    """

    # Save the summary to a markdown file
    with open('campaign_summary.md', 'w') as file:
        file.write(summary)

    print("Campaign summary report generated and saved to 'campaign_summary.md'!")

# Example usage
csv_file_path = '/content/updated_leads_with_status.csv'
generate_campaign_summary(csv_file_path)

Campaign summary report generated and saved to 'campaign_summary.md'!


In [7]:
!cat campaign_summary.md


    # Campaign Summary

    - **Total Leads Processed**: 25
    - **Emails Sent Successfully**: 25
    - **Emails Failed**: 0

    ## Key Insights:
    - Most of the emails were successfully sent, with only a few failures.
    - Consider retrying the failed emails or investigating the issues (e.g., invalid email addresses, network issues).

    ## Recommendations:
    - Review the failed email logs to ensure they are delivered successfully in the next campaign.
    